In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
import time
from ultralytics import YOLO
from torchvision import models, transforms
from timm.models import convnext_small
import torch.nn as nn
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
from datetime import datetime
import logging
from tqdm import tqdm
import functools

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
FRAME_INTERVAL = 1
BATCH_SIZE = 200
SURFACE_CATEGORIES = {
    "dry_asphalt_slight": "Easy to Drive", "dry_asphalt_smooth": "Easy to Drive",
    "dry_concrete_slight": "Easy to Drive", "dry_concrete_smooth": "Easy to Drive",
    "dry_gravel": "Easy to Drive", "wet_asphalt_smooth": "Easy to Drive",
    "wet_concrete_smooth": "Easy to Drive", "wet_gravel": "Easy to Drive",
    "wet_mud": "Easy to Drive", "dry_asphalt_severe": "Take Precautions",
    "dry_concrete_severe": "Take Precautions", "water_asphalt_smooth": "Take Precautions",
    "water_concrete_smooth": "Take Precautions", "water_gravel": "Take Precautions",
    "water_mud": "Take Precautions", "wet_asphalt_slight": "Take Precautions",
    "wet_concrete_slight": "Take Precautions", "ice": "Dangerous",
    "fresh_snow": "Dangerous", "melted_snow": "Dangerous",
    "water_asphalt_severe": "Dangerous", "water_asphalt_slight": "Dangerous",
    "water_concrete_severe": "Dangerous", "water_concrete_slight": "Dangerous",
    "wet_asphalt_severe": "Dangerous", "wet_concrete_severe": "Dangerous"
}
ROAD_ONLY_CLASSES = ['animal', 'human', 'vehicle', 'speedbump', 'obstacle']
ANYWHERE_CLASSES = ['road sign', 'go', 'stop', 'slowdown']
VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle']

class IntegratedTrafficSystem:
    def __init__(self, device='cpu'):
        self.device = torch.device(device)
        self.frame_cache = {}
        self.load_models_with_error_handling()
        # Parameters for traffic recommendation using Greenshields model
        self.L = 1000.0          # Segment length in meters (1 km)
        self.v_f = 16.67         # Free-flow speed in m/s (60 km/h)
        self.k_j = 0.1           # Jam density in vehicles/m (1 vehicle per 10 m)
        self.v_min = 5.0         # Minimum speed in m/s (18 km/h)
        self.time_savings_threshold = 5.0  # Minimum time savings in seconds for lane change

    def load_models_with_error_handling(self):
        try:
            self.load_dashcam_models()
            self.load_traffic_models()
        except Exception as e:
            logging.error(f"Model loading failed: {e}")
            raise

    @functools.lru_cache(maxsize=128)
    def load_dashcam_models(self):
        logging.info("Loading dashcam models...")
        try:
            self.object_model = YOLO("/content/drive/MyDrive/Colab Notebooks/best.pt").to(self.device)
            self.seg_model = YOLO("/content/drive/MyDrive/Colab Notebooks/road_seg (2).pt").to(self.device)

            self.weather_model = models.efficientnet_b0(weights=None)
            num_ftrs = self.weather_model.classifier[1].in_features
            self.weather_model.classifier[1] = nn.Linear(num_ftrs, 3)
            self.weather_model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/weather_model.pth", map_location=self.device))
            self.weather_model.to(self.device).eval()
            self.weather_classes = ["Clear", "Fog", "Rain"]
            self.weather_transform = transforms.Compose([
                transforms.Resize((224, 224)), transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

            checkpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/final_model.pth", map_location=self.device)
            self.surface_model = convnext_small(pretrained=False)
            in_features = self.surface_model.head.fc.in_features
            self.surface_model.head.fc = nn.Linear(in_features, len(checkpoint['class_mapping']))
            self.surface_model.load_state_dict(checkpoint['model_state_dict'], strict=False)
            self.surface_model.to(self.device).eval()
            self.surface_class_mapping = checkpoint['class_mapping']
            self.surface_transform = transforms.Compose([
                transforms.ToPILImage(), transforms.Resize(256), transforms.CenterCrop(224),
                transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            logging.info("Dashcam models loaded successfully.")
        except Exception as e:
            logging.error(f"Failed to load dashcam models: {e}")
            raise

    def load_traffic_models(self):
        logging.info("Loading traffic models...")
        try:
            self.traffic_detection_model = YOLO("yolo11n.pt").to(self.device)
            self.scaler = joblib.load("/content/drive/MyDrive/Colab Notebooks/LSTM/scaler.pkl")
            self.le_day = joblib.load("/content/drive/MyDrive/Colab Notebooks/LSTM/label_encoder_day.pkl")
            self.traffic_models = [
                load_model(f"/content/drive/MyDrive/Colab Notebooks/LSTM/lstm_traffic_model_{i}.h5", compile=False)
                for i in range(3)
            ]
            for model in self.traffic_models:
                model.compile(optimizer='adam', loss='mse')
            self.global_counted_vehicles = set()
            logging.info("Traffic models loaded successfully.")
        except Exception as e:
            logging.error(f"Failed to load traffic models: {e}")
            raise

    def get_road_mask(self, frame):
        results = self.seg_model(frame, task='segment', verbose=False)
        road_mask = None
        for result in results:
            for i, cls in enumerate(result.boxes.cls.cpu().numpy()):
                if cls == 0 and len(result.masks) > i:
                    mask = result.masks[i].data.cpu().numpy()[0]
                    road_mask = mask if road_mask is None else np.logical_or(road_mask, mask)
        return cv2.resize(road_mask.astype(np.uint8) * 255, (frame.shape[1], frame.shape[0])) if road_mask is not None else None

    def is_near_road(self, box, road_mask, threshold=30):
        if road_mask is None:
            return False
        x1, y1, x2, y2 = box.astype(int)
        center_x, bottom_y = (x1 + x2) // 2, min(y2, road_mask.shape[0] - 1)
        center_x = min(center_x, road_mask.shape[1] - 1)
        if road_mask[bottom_y, center_x] > 0:
            return True
        dist_transform = cv2.distanceTransform(255 - road_mask, cv2.DIST_L2, 3)
        return dist_transform[bottom_y, center_x] < threshold

    def determine_vehicle_proximity(self, box, frame_height, frame_width):
        x1, y1, x2, y2 = box.astype(int)
        box_height = y2 - y1
        box_width = x2 - x1
        box_area = box_height * box_width
        frame_area = frame_height * frame_width
        relative_size = box_area / frame_area
        position_factor = y2 / frame_height
        proximity_score = relative_size * 0.7 + position_factor * 0.3
        return "CLOSE" if proximity_score > 0.05 else "FAR"

    def preprocess_traffic_input(self, timestamp_str, day, red_light, left, right, lookback=10):
        timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
        day_encoded = self.le_day.transform([day])[0]
        red_light_encoded = 1 if red_light.lower() == "yes" else 0
        red_light_importance = red_light_encoded * 3
        input_data = np.array([left, right, day_encoded, red_light_encoded, red_light_importance])
        sequence = np.tile(input_data, (lookback, 1))
        dummy_target = np.zeros((lookback, 2))
        sequence_with_dummy = np.hstack((sequence, dummy_target))
        scaled_sequence = self.scaler.transform(sequence_with_dummy)
        return np.expand_dims(scaled_sequence[:, :len(input_data)], axis=0)

    def apply_traffic_adjustments(self, timestamp_str, day, red_light, pred):
        timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
        hour, minute = timestamp.hour, timestamp.minute
        left, right = pred
        if day in ["Saturday", "Sunday"]:
            if red_light.lower() == "yes":
                #left += np.random.randint(1, 3)
                right += np.random.randint(12, 15)
        else:
            if red_light.lower() == "yes":
                #left += np.random.randint(3, 6)
                right += np.random.randint(12, 15)
            elif 15 <= hour <= 17:
                #left += np.random.randint(2, 4)
                right += np.random.randint(12, 15)
        return left, right

    def predict_traffic(self, timestamp_str, day, red_light, left, right):
        logging.info(f"Predicting traffic with left={left}, right={right}")
        try:
            X_test = self.preprocess_traffic_input(timestamp_str, day, red_light, left, right)
            predictions = [model.predict(X_test, verbose=0) for model in self.traffic_models]
            pred_scaled = np.mean(predictions, axis=0)
            pred_with_dummy = np.hstack((np.zeros((1, 5)), pred_scaled))
            pred = self.scaler.inverse_transform(pred_with_dummy)[:, 5:]
            left_pred, right_pred = self.apply_traffic_adjustments(timestamp_str, day, red_light, (pred[0][0], pred[0][1]))
            logging.info(f"Traffic prediction: left={left_pred:.1f}, right={right_pred:.1f}")
            return left_pred, right_pred
        except Exception as e:
            logging.error(f"Error in traffic prediction: {e}")
            return None, None

    def process_cctv_frame(self, frame, width, left_vehicles, right_vehicles):
        output_frame = frame.copy()
        mid_point = width // 2
        results = self.traffic_detection_model.track(frame, persist=True, device=self.device, classes=[2, 3, 5, 7])

        for box in results[0].boxes:
            if box.id is not None:
                class_name = self.traffic_detection_model.names[int(box.cls)]
                if class_name in VEHICLE_CLASSES:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    vehicle_id = int(box.id)
                    center_x = (x1 + x2) / 2
                    color = (0, 255, 0) if vehicle_id in self.global_counted_vehicles else (255, 0, 0)
                    cv2.rectangle(output_frame, (x1, y1), (x2, y2), color, 2)
                    label = f"ID:{vehicle_id} ({class_name})" + (" ✓" if vehicle_id in self.global_counted_vehicles else "")
                    cv2.putText(output_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    if center_x < mid_point and vehicle_id not in self.global_counted_vehicles:
                        left_vehicles.add(vehicle_id)
                        self.global_counted_vehicles.add(vehicle_id)
                    elif center_x >= mid_point and vehicle_id not in self.global_counted_vehicles:
                        right_vehicles.add(vehicle_id)
                        self.global_counted_vehicles.add(vehicle_id)

        cv2.line(output_frame, (mid_point, 0), (mid_point, frame.shape[0]), (0, 0, 255), 2)
        cv2.putText(output_frame, f"Left: {len(left_vehicles)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(output_frame, f"Right: {len(right_vehicles)}", (width - 200, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        return output_frame

    def process_dashcam_frame(self, frame):
        frame_hash = hash(frame.tobytes())
        if frame_hash in self.frame_cache:
            return self.frame_cache[frame_hash]

        road_mask = self.get_road_mask(frame)
        result_frame = frame.copy()
        height, width = frame.shape[:2]

        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        weather_input = self.weather_transform(pil_image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            weather_output = self.weather_model(weather_input)
            weather_pred = self.weather_classes[torch.max(weather_output, 1)[1].item()]

        surface_category = "Unknown"
        if road_mask is not None:
            mask_3channel = np.stack([road_mask] * 3, axis=2) // 255
            masked_frame = frame * mask_3channel
            surface_input = self.surface_transform(cv2.cvtColor(masked_frame, cv2.COLOR_BGR2RGB)).unsqueeze(0).to(self.device)
            with torch.no_grad():
                surface_output = self.surface_model(surface_input)
                surface_pred = self.surface_class_mapping[torch.max(surface_output, 1)[1].item()]
                surface_category = SURFACE_CATEGORIES.get(surface_pred, "Unknown")

        results = self.object_model(frame, conf=0.5, iou=0.45, verbose=False)
        detected = {'speedbump': False, 'stop': False, 'slowdown': False, 'left': 0, 'right': 0}
        color_palette = np.random.randint(0, 255, size=(len(self.object_model.names), 3)).tolist()

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            class_ids = result.boxes.cls.cpu().numpy().astype(int)
            for box, conf, cls_id in zip(boxes, confidences, class_ids):
                class_name = self.object_model.names[cls_id]
                if class_name in ROAD_ONLY_CLASSES and (road_mask is None or not self.is_near_road(box, road_mask)):
                    continue
                x1, y1, x2, y2 = box.astype(int)
                center_x = (x1 + x2) // 2
                color = tuple(map(int, color_palette[cls_id]))
                cv2.rectangle(result_frame, (x1, y1), (x2, y2), color, 2)
                proximity = self.determine_vehicle_proximity(box, height, width) if class_name == 'vehicle' else ""
                label = f"{class_name} {conf:.2f}" + (f" - {proximity}" if proximity else "")
                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
                cv2.rectangle(result_frame, (x1, y1 - h - 10), (x1 + w, y1), color, -1)
                cv2.putText(result_frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

                if class_name == 'speedbump':
                    detected['speedbump'] = True
                elif class_name == 'stop':
                    detected['stop'] = True
                elif class_name == 'slowdown':
                    detected['slowdown'] = True
                elif class_name in ['human', 'animal', 'vehicle', 'obstacle']:
                    if center_x < width // 2:
                        detected['left'] += 1
                    else:
                        detected['right'] += 1

        result = (result_frame, weather_pred, surface_category, detected)
        self.frame_cache[frame_hash] = result
        return result

    def calculate_risk_score(self, weather, surface, detected):
        score = 0
        if weather in ["Fog", "Rain"]:
            score += 30
        if surface == "Dangerous":
            score += 40
        elif surface == "Take Precautions":
            score += 20
        score += (detected['left'] + detected['right']) * 5
        if detected['speedbump'] or detected['stop'] or detected['slowdown']:
            score += 15
        return min(score, 100)

    def get_recommendation(self, detected, left_pred, right_pred, timestamp_str):
        """
        Generate driving and traffic recommendations.
        Traffic recommendation uses Greenshields model to estimate travel times.
        """
        # Basic driving recommendation based on detected objects
        rec = ""
        if detected['speedbump'] and (detected['stop'] or detected['slowdown']):
            rec = "Watch for people"
        elif detected['speedbump']:
            rec = "Go slow"
        elif detected['stop'] or detected['slowdown']:
            rec = "Watch for people and cross"
        elif detected['left'] > 0 or detected['right'] > 0:
            if detected['left'] == detected['right']:
                rec = "Go slow"
            elif detected['left'] > detected['right']:
                rec = "Go slow but right"
            else:
                rec = "Go slow but left"
        else:
            rec = "Go fast"
        if any(class_name in self.object_model.names for class_name in ANYWHERE_CLASSES):
            rec += " [read the sign]"

        # Traffic recommendation using Greenshields model
        traffic_rec = "Stay in lane"
        lane_change_timing = ""
        time_savings = 0.0
        if left_pred is not None and right_pred is not None:
            # Calculate traffic densities (vehicles per meter)
            k_left = left_pred / self.L
            k_right = right_pred / self.L

            # Calculate speeds using Greenshields model with minimum speed constraint
            v_left = max(self.v_f * (1 - k_left / self.k_j), self.v_min)
            v_right = max(self.v_f * (1 - k_right / self.k_j), self.v_min)

            # Calculate travel times over the segment
            t_left = self.L / v_left
            t_right = self.L / v_right

            # Recommend lane change if time savings exceed threshold
            if t_right < t_left - self.time_savings_threshold:
                traffic_rec = "Move to right lane"
                lane_change_timing = "in 10-15s"
                time_savings = t_left - t_right
            elif t_left < t_right - self.time_savings_threshold:
                traffic_rec = "Move to left lane"
                lane_change_timing = "in 10-15s"
                time_savings = t_right - t_left

        # Construct traffic recommendation string
        traffic_part = f"Traffic in 60s: {traffic_rec}"
        if lane_change_timing:
            traffic_part += f" {lane_change_timing} (Save ~{time_savings:.1f}s)"

        # Combine basic and traffic recommendations
        return f"{rec} | {traffic_part}"

    def process_videos_alternating(self, dashcam_path, cctv_path, timestamp_str, day, red_light):
        logging.info("Starting alternating video processing...")
        try:
            cctv_cap = cv2.VideoCapture(cctv_path)
            dashcam_cap = cv2.VideoCapture(dashcam_path)

            if not cctv_cap.isOpened() or not dashcam_cap.isOpened():
                logging.error("Failed to open one or both videos.")
                return None, None

            cctv_fps = cctv_cap.get(cv2.CAP_PROP_FPS)
            cctv_width = int(cctv_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            cctv_height = int(cctv_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            dashcam_fps = dashcam_cap.get(cv2.CAP_PROP_FPS)
            dashcam_width = int(dashcam_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            dashcam_height = int(dashcam_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            cctv_out = cv2.VideoWriter("cctv_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), cctv_fps, (cctv_width, cctv_height))
            dashcam_out = cv2.VideoWriter("dashcam_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), dashcam_fps, (dashcam_width, dashcam_height))

            frame_count = 0
            left_vehicles, right_vehicles = set(), set()
            left_pred, right_pred = None, None

            total_frames = min(BATCH_SIZE + int(dashcam_cap.get(cv2.CAP_PROP_FRAME_COUNT)),
                             int(cctv_cap.get(cv2.CAP_PROP_FRAME_COUNT)))
            progress_bar = tqdm(total=total_frames, desc="Processing Frames")

            while frame_count < BATCH_SIZE and cctv_cap.isOpened() and dashcam_cap.isOpened():
                ret_cctv, cctv_frame = cctv_cap.read()
                if ret_cctv:
                    cctv_result_frame = self.process_cctv_frame(cctv_frame, cctv_width, left_vehicles, right_vehicles)
                    cctv_out.write(cctv_result_frame)

                ret_dashcam, dashcam_frame = dashcam_cap.read()
                if ret_dashcam:
                    dashcam_result_frame, weather, surface, detected = self.process_dashcam_frame(dashcam_frame)
                    risk_score = self.calculate_risk_score(weather, surface, detected)
                    recommendation = self.get_recommendation(detected, left_pred, right_pred, timestamp_str)
                    overlay = dashcam_result_frame.copy()
                    cv2.rectangle(overlay, (0, 0), (dashcam_width, 140), (0, 0, 0), cv2.FILLED)  # Increased overlay height
                    cv2.addWeighted(overlay, 0.6, dashcam_result_frame, 0.4, 0, dashcam_result_frame)

                    # Increased font sizes for all text elements
                    cv2.putText(dashcam_result_frame, f"Weather: {weather}", (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
                    cv2.putText(dashcam_result_frame, f"Surface: {surface}", (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
                    cv2.putText(dashcam_result_frame, f"Risk Score: {risk_score}%", (10, 115), cv2.FONT_HERSHEY_SIMPLEX, 1.2,
                               (0, 255, 0) if risk_score < 50 else (0, 0, 255), 3)

                    # Add background to make AI Rec more readable
                    overlay_bottom = dashcam_result_frame.copy()
                    cv2.rectangle(overlay_bottom, (0, dashcam_height - 60), (dashcam_width, dashcam_height), (0, 0, 0), cv2.FILLED)
                    cv2.addWeighted(overlay_bottom, 0.7, dashcam_result_frame, 0.3, 0, dashcam_result_frame)

                    # Increased font size for AI Recommendation at the bottom
                    cv2.putText(dashcam_result_frame, f"AI Rec: {recommendation}", (10, dashcam_height - 20),
                               cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 0), 3)

                    dashcam_out.write(dashcam_result_frame)

                frame_count += 1
                progress_bar.update(1)

            if frame_count >= BATCH_SIZE:
                left_count, right_count = len(left_vehicles), len(right_vehicles)
                logging.info(f"CCTV stopped at 200 frames. Current: Left={left_count}, Right={right_count}")
                left_pred, right_pred = self.predict_traffic(timestamp_str, day, red_light, left_count, right_count)
                cctv_cap.release()
                cctv_out.release()

            while dashcam_cap.isOpened():
                ret, frame = dashcam_cap.read()
                if not ret:
                    break
                result_frame, weather, surface, detected = self.process_dashcam_frame(frame)
                risk_score = self.calculate_risk_score(weather, surface, detected)
                recommendation = self.get_recommendation(detected, left_pred, right_pred, timestamp_str)

                # Create top overlay for info display
                overlay = result_frame.copy()
                cv2.rectangle(overlay, (0, 0), (dashcam_width, 170), (0, 0, 0), cv2.FILLED)  # Increased overlay height
                cv2.addWeighted(overlay, 0.6, result_frame, 0.4, 0, result_frame)

                # Increased font sizes for all text elements at the top
                cv2.putText(result_frame, f"Weather: {weather}", (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
                cv2.putText(result_frame, f"Surface: {surface}", (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
                cv2.putText(result_frame, f"Risk Score: {risk_score}%", (10, 115), cv2.FONT_HERSHEY_SIMPLEX, 1.2,
                           (0, 255, 0) if risk_score < 50 else (0, 0, 255), 3)

                if left_pred is not None and right_pred is not None:
                    cv2.putText(result_frame, f"Pred: Left={left_pred:.1f}, Right={right_pred:.1f}",
                               (10, 155), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 3)

                # Add background to make AI Rec more readable
                overlay_bottom = result_frame.copy()
                cv2.rectangle(overlay_bottom, (0, dashcam_height - 60), (dashcam_width, dashcam_height), (0, 0, 0), cv2.FILLED)
                cv2.addWeighted(overlay_bottom, 0.7, result_frame, 0.3, 0, result_frame)

                # Increased font size for AI Recommendation at the bottom
                cv2.putText(result_frame, f"AI Rec: {recommendation}", (10, dashcam_height - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 0), 3)

                dashcam_out.write(result_frame)
                frame_count += 1
                progress_bar.update(1)
                if frame_count % 50 == 0:
                    logging.info(f"Processed dashcam frame {frame_count}")

            progress_bar.close()
            dashcam_cap.release()
            dashcam_out.release()
            logging.info("Video processing completed.")
            return "dashcam_output.mp4", "cctv_output.mp4"

        except Exception as e:
            logging.error(f"Error in video processing: {e}")
            return None, None
def main():
    try:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        system = IntegratedTrafficSystem(device=device)
        dashcam_out, cctv_out = system.process_videos_alternating(
            "/content/drive/MyDrive/Colab Notebooks/1666547-hd_1920_1080_30fps.mp4",
            "/content/drive/MyDrive/Colab Notebooks/new_cctv.mp4",
            "2025-04-03 15:03:00",
            "Thursday",
            "yes"
        )
        print(f"Dashcam output saved as: {dashcam_out}")
        print(f"CCTV output saved as: {cctv_out}")
    except Exception as e:
        logging.error(f"Main execution failed: {e}")
        print("An error occurred. Check logs for details.")

if __name__ == "__main__":
    main()